### 녹음을 위한 패키지 설치

In [1]:
!pip install sounddevice
!pip install soundfile

#### 음성 녹음 test

In [2]:
import time
import queue, os, threading
import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write

q = queue.Queue()
recorder = False
recording = False

def complicated_record():
    with sf.SoundFile("test.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())
        
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())
    
def start():
    global recorder
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('start recording')
    recorder.start()
    
def stop():
    global recorder
    global recording
    recording = False
    recorder.join()
    print('stop recording')
    
start()
time.sleep(5)
stop()

start recording
stop recording


### 1단계: 카카오 개발자 페이지 접속하기
1. 카카오 디벨로퍼스에 접속합니다.(https://developers.kakao.com)
2. 오른쪽 상단의 로그인 버튼을 클릭해 카카오 계정으로 로그인 합니다. *카카오 계정이 없는 경우 새롭게 계정을 생성합니다.

### 2단계: 내 애플리케이션 만들기
1. 상단 메뉴에서 내 애플리케이션을 클릭합니다.
2. 서비스 이용 동의 페이지가 나타나는 경우 필수 항목에 동의하고, 개발자 이름을 입력한 다음 회원가입 버튼을 클 릭해 다음 단계로 넘어갑니다. (기존에 서비스 이용 동의한 경우 표시되지 않을 수 있습니다.)
3. 전체 애플리케이션 페이지가 나타나면, 애플리케이션 추가하기를 클릭합니다.
4. 앱 아이콘(로고)을 업로드하고, 앱 이름, 회사 이름을 작성합니다.
5. 저장 버튼을 클릭합니다.
6. REST API 키 복사
7. 왼쪽 메뉴중 음성 메뉴를 선택후 상태를 On으로 바꿔야 됨

#### STT 테스트

In [3]:
import requests
import json
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

rest_api_key = '954052c06ac962dca9151a748fdd9289'

headers = {
    "Content-Type": "application/octet-stream",
    "X-DSS-Service": "DICTATION",
    "Authorization": "KakaoAK " + rest_api_key,
}

with open('test.wav', 'rb') as fp:
    audio = fp.read()

res = requests.post(kakao_speech_url, headers=headers, data=audio)
print(res.text)

------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"beginPointDetection","value":"BPD"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 개발자"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 개발자 베이지"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 개발자 페이지 접속"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8

{"type":"endPointDetection","value":"EPD"}
------newtoneeij2hSTub8Iqihmj
Content-Type: application/json; charset=UTF-8
Speech-Length: 4

{"type":"finalResult","value":"카카오 개발자 페이지 접속하니","nBest":[{"value":"카카오 개발자 페이지 접속하니","score":25},{"value":"카카오 개발자 페이지 접속하기","score":0},{"value":"카카오 개발자 베이지 접속하니","score":0},{

#### finalResult 추출 방법

In [4]:
result_json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(result_json_string)
# print(result)
print(result['value'])

카카오 개발자 페이지 접속하니


#### TTS 테스트

In [5]:
import os, requests, json, wave
url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = '954052c06ac962dca9151a748fdd9289'
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}

data = "<speak>카카오 테스트중 입니다.</speak>".encode('utf-8').decode('latin1')
res = requests.post(url, headers=headers, data=data)
f= open('temp.wav', 'wb')
f.write(res.content)
f.close()